In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Getting HTML content for news listing page
soup = make_request(browser, '/archivo/politica/')

In [8]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [9]:
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)
print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion

3


In [10]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
len(cards)

8

In [11]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [12]:
news

[{'title': '‘Se enfrenta un blanco legítimo’: Duque sobre bombardeo en el que murieron menores',
  'relative_path': '/politica/se-enfrenta-un-blanco-legitimo-duque-sobre-bombardeo-en-el-que-murieron-menores/'},
 {'title': 'Se cae proyecto que creaba bonificación adicional para congresistas',
  'relative_path': '/politica/se-cae-proyecto-que-creaba-bonificacion-adicional-para-congresistas/'},
 {'title': '“Salimos del cascarón del olvido”: hermanos Korebajʉ',
  'relative_path': '/politica/salimos-del-cascaron-del-olvido-hermanos-korebaj/'},
 {'title': 'Nuevas alertas en Ley de Presupuesto: desfinanciaría a Mi Casa Ya',
  'relative_path': '/politica/nuevas-alertas-en-ley-de-presupuesto-desfinanciaria-a-mi-casa-ya/'},
 {'title': 'Coalición de la Esperanza respalda consulta popular a jóvenes en diciembre',
  'relative_path': '/politica/coalicion-de-la-esperanza-respalda-consulta-popular-a-jovenes-en-diciembre/'},
 {'title': 'Alejandro Gaviria, ¿manzana de discordia entre Coalición de la Esp

In [13]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [14]:
news

[{'title': '‘Se enfrenta un blanco legítimo’: Duque sobre bombardeo en el que murieron menores',
  'relative_path': '/politica/se-enfrenta-un-blanco-legitimo-duque-sobre-bombardeo-en-el-que-murieron-menores/',
  'datetime': '8 Oct 2021  - 11:55 a.\xa0m.',
  'author': 'Redacción Política',
  'summary': 'Aunque el primer mandatario resaltó que se trata de hechos que le duelen al país, defendió que es deber de la Fuerza Pública actuar sobre los criminales y con base en estándares internacionales.',
  'full_text': 'En medio de la controversia por la muerte de cuatro menores de edad en medio de un bombardeo contra el Eln en Chocó, este viernes el presidente Iván Duque se refirió a la polémica y aseguró que “se está enfrentando a un blanco legítimo”. En contexto: Vuelve y juega: en bombardeo al Eln en Chocó murieron cuatro menores de edad El mandatario, de visita en Amazonas –donde instaló la Tercera Cumbre del Pacto de Leticia–, defendió que en el operativo no había civiles y destacó que en

In [15]:
# Storing extracted information for further analysis
collection.insert_many(news)